In [ ]:
import os
import torch

X = []
Y = []

for label in os.listdir('datasetzoo'):
  label_path = os.path.join('datasetzoo', label)
  for tensor_file in os.listdir(label_path):
    file_path = os.path.join(label_path, tensor_file)
    tensor = torch.load(file_path)
    X.append(tensor)
    Y.append(label)

In [ ]:
import os
import torch
import numpy as np

X = []
Y = []

unique_labels = list(set(labels))

for tensor_file in tensor_files:
  tensor = torch.load(tensor_file)
  X.append(tensor)
  label = os.path.dirname(tensor_file)
  one_hot = np.zeros(len(unique_labels))
  one_hot[unique_labels.index(label)] = 1
  Y.append(one_hot)

In [ ]:
import os
import torch
import numpy as np

X = []
Y = []
unique_labels = os.listdir('datasetzoo')

for label in unique_labels:
  label_path = os.path.join('datasetzoo', label)
  for tensor_file in os.listdir(label_path):
    file_path = os.path.join(label_path, tensor_file)
    tensor = torch.load(file_path)
    X.append(tensor)
    # One-hot encode 
    one_hot = np.zeros(len(unique_labels))
    one_hot[unique_labels.index(label)] = 1
    Y.append(one_hot)

# Convert Y from a list of one-hot encoded labels to a numpy array
Y = np.array(Y)

In [ ]:
import os
import torch


base_directory = 'signs/'

# Lists to store the tensors and labels
tensors = []
labels = []

# Loop through all the class directories
for class_name in os.listdir(base_directory):
  class_directory = os.path.join(base_directory, class_name)
  
  for filename in os.listdir(class_directory):
    # Load the tensor from the file
    tensor = torch.load(os.path.join(class_directory, filename))
    
    # Add the tensor and label to the lists
    tensors.append(tensor)
    labels.append(class_name)

In [ ]:
import glob
import torch
from sklearn.model_selection import train_test_split

file_paths = glob.glob('datasetz/*.pt')

tensors = []

class_names = []

for file_path in file_paths:
    class_name = file_path.split('_')[0]
    
    tensor_list = torch.load(file_path)
    
    # add the tensors and class names to the respective lists
    tensors.extend(tensor_list)
    class_names.extend([class_name] * len(tensor_list))

# split the tensors into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tensors, class_names, test_size=0.2)

In [ ]:
import os
import torch

X = []

for label in labels:
  samples = []
  for tensor_file in os.listdir(label):
    tensor = torch.load(os.path.join(label, tensor_file))
    samples.append(tensor)
  X.append(samples)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
#Test tensor size
import torch

# Load the tensor from the file
tensor = torch.load('datasetz/bye_aug_1.pt')

# Check the shape of the tensor
print(tensor.shape)

In [ ]:
#Doesn't work
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()

        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.identity = nn.Identity()
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.identity(x)
        x, _ = self.lstm2(x)
        x = self.identity(x)
        x, _ = self.lstm3(x)
        x = self.identity(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

model = LSTMModel(input_dim=258, hidden_dim=64, output_dim=signs.shape[0])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(2000):
    # Forward pass
    y_pred = model(X_train)

    # Print loss
    loss = loss_fn(y_pred, y_train)
    print('Epoch: ', epoch, 'Loss: ', loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
#issues with tensorboard, give up for now
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.relu1 = nn.ReLU()
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.relu2 = nn.ReLU()
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.relu3 = nn.ReLU()
        self.fc1 = nn.Linear(hidden_size, 64, activation=nn.ReLU())
        self.fc2 = nn.Linear(64, 32, activation=nn.ReLU())
        self.fc3 = nn.Linear(32, output_size)
        
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.relu1(x)
        x, _ = self.lstm2(x)
        x = self.relu2(x)
        x, _ = self.lstm3(x)
        x = self.relu3(x)
        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
    
# Define model, loss function, and optimizer
model = LSTMModel(258, signs.shape[0], 64)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Define a tensorboard logger
log_dir = os.path.join('Logs')
tb_logger = SummaryWriter(log_dir)

# Train the model
for epoch in range(num_epochs):
    # ...
    # Calculate loss and accuracy
    # ...
    tb_logger.add_scalar('Loss/train', loss.item(), epoch)
    tb_logger.add_scalar('Accuracy/train', accuracy, epoch)

# Save the model
torch.save(model.state_dict(), 'model.pt')

In [ ]:
#Minimum viable model so far
#Mistake was not converting to float32
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()

        self.lstm = nn.LSTM(input_size, lstm_hidden_size, num_rnn_layers, batch_first=True) #bidirectional=True, dropout= ???
        self.fc1 = nn.Linear(lstm_hidden_size, fc_hidden_size)
        self.fc2 = nn.Linear(fc_hidden_size, fc_hidden_size)
        self.fc3 = nn.Linear(fc_hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.softmax = nn.Softmax()


    def forward(self, x):
        # Define initial tensors for hidden and cell states
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_lstm_layers, batch_size, self.lstm_hidden_size).to(device) 
        c0 = torch.zeros(self.num_lstm_layers, batch_size, self.lstm_hidden_size).to(device) 

        # Pass input with initial tensors to lstm layers
        outlstm,  = self.lstm(x, (h0, c0))
        out_relu = self.relu(out_lstm)

        # Pass last timestep to fc layers
        in_fc1 = out_relu[:, -1, :]
        out_fc1 = self.fc1(in_fc1)
        in_fc2 = self.relu(out_fc1)

        out_fc2 = self.fc2(in_fc2)
        in_fc3 = self.relu(out_fc1)

        out_fc3 = self.fc3(in_fc3)
        out = self.softmax(out_fc3)

        return out

num_rnn_layers = 3
model = LSTMModel(input_dim=258, hidden_dim=64, output_dim=signs.shape[0])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Split the dataset with sci-kit learn before this, categorical split
X_train = X_train.to(torch.float32)
y_train = y_train.to(torch.long)

In [ ]:
# Timestep
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        out = self.fc1(out[:, -1, :])
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

model = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)

# Set the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#Simplify
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()

        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.identity = nn.Identity()
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.identity(x)
        x, _ = self.lstm2(x)
        x = self.identity(x)
        x, _ = self.lstm3(x)
        x = self.identity(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

model = LSTMModel(input_dim=258, hidden_dim=64, output_dim=signs.shape[0])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

for epoch in range(1000):
    # Forward pass
    y_pred = model(X_train)

    # Print loss
    loss = loss_fn(y_pred, y_train)
    print('Epoch: ', epoch, 'Loss: ', loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class LSTMModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, output_size)
        
    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x
    
# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Loop through the training data for a specified number of epochs
for epoch in range(num_epochs):
    # Loop through the individual training examples
    for inputs, labels in training_data:
        # Clear the gradients
        optimizer.zero_grad()
        
        # Forward pass: pass the inputs through the model to get the predicted output
        outputs = model(inputs)
        
        # Calculate the loss
        loss = loss_fn(outputs, labels)
        
        # Backward pass: compute the gradients of the loss with respect to the model's parameters
        loss.backward()
        
        # Update the model's parameters
        optimizer.step()

# Save the trained model
torch.save(model.state_dict(), 'model.pt')

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()

        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.identity = nn.Identity()
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.identity(x)
        x, _ = self.lstm2(x)
        x = self.identity(x)
        x, _ = self.lstm3(x)
        x = self.identity(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

model = LSTMModel(input_dim=258, hidden_dim=64, output_dim=signs.shape[0])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

for epoch in range(1000):
    # Forward pass
    y_pred = model(X_train)

    # Loss and accuracy
    loss = loss_fn(y_pred, y_train)
    _, prediction = y_pred.max(dim=1)
    accuracy = (prediction == y_train).float().mean()

    # Print loss and accuracy
    print('Epoch: ', epoch, 'Loss: ', loss.item(), 'Accuracy: ', accuracy.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
#Minimum viable model so far
#Add relu to it
#Mistake was not converting to float32
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()

        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.identity = nn.Identity()
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.identity(x)
        x, _ = self.lstm2(x)
        x = self.identity(x)
        x, _ = self.lstm3(x)
        x = self.identity(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

model = LSTMModel(input_dim=258, hidden_dim=64, output_dim=signs.shape[0])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

for epoch in range(2000):
    # Forward pass
    y_pred = model(X_train)

    # Loss and accuracy
    loss = loss_fn(y_pred, y_train)
    _, prediction = y_pred.max(dim=1)
    accuracy = (prediction == y_train).float().mean()

    # Print loss and accuracy
    print('Epoch: ', epoch, 'Loss: ', loss.item()*100, '%', 'Accuracy: ', accuracy.item()*100, '%')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()

        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.identity = nn.Identity()
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.identity(x)
        x, _ = self.lstm2(x)
        x = self.identity(x)
        x, _ = self.lstm3(x)
        x = self.identity(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

model = LSTMModel(input_dim=258, hidden_dim=64, output_dim=signs.shape[0])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Create a new tensor for X_train that shares the same memory and does not require gradient calculations
X_train = X_train.clone().detach()

# Convert X_train to a tensor with dtype torch.float32
X_train = X_train.type(torch.float32)

# Create a new tensor for y_train that shares the same memory and does not require gradient calculations
y_train = y_train.clone().detach()

# Convert y_train to a tensor with dtype torch.long
y_train = y_train.type(torch.long)

# Reshape X_train to have a shape of (batch_size, sequence_length, input_dim)
X_train = X_train.reshape(-1, 30, 258)

for epoch in range(1000):
    # Forward pass
    y_pred = model(X_train)

    # Compute loss and accuracy
    loss = loss_fn(y_pred, y_train)
    _, prediction = y_pred.max(dim=1)
    accuracy = (prediction == y_train).float().mean()

    # Print loss and accuracy
    print(f'Epoch: {epoch+1:03d} | Loss: {loss:.5f} | Accuracy: {accuracy:.5f}')

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()

        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x


model = LSTMModel(input_dim=258, hidden_dim=64, output_dim=signs.shape[0])


X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
model = torch.load('model.pth')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# TensorBoard writer
writer = tb.SummaryWriter()

for epoch in range(2000):
    # Train the model on one epoch
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the validation set
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        # Write the loss to TensorBoard
        writer.add_scalar('val_loss', loss, epoch)

# Close the TensorBoard writer
writer.close()

In [ ]:
# Evaluate the model on the test data
X_test = X_test.clone().detach()
y_test = y_test.clone().detach()

# Forward pass
y_pred = model(X_test)

# Compute loss and accuracy
test_loss = loss_fn(y_pred, y_test)
_, prediction = y_pred.max(dim=1)
test_accuracy = (prediction == y_test).float().mean()

print('Test loss: ', test_loss.item()*100, '%', 'Test accuracy: ', test_accuracy.item()*100, '%')

In [ ]:
import optuna

def optimize(trial):
    input_dim = 258
    hidden_dim = trial.suggest_int('hidden_dim', 16, 128)
    output_dim = signs.shape[0]

    model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=trial.suggest_loguniform('lr', 1e-5, 1e-1))
  
    for epoch in range(500):
        # Forward pass
        y_pred = model(X_train)

        # Compute loss and accuracy
        loss = loss_fn(y_pred, y_train)
        _, prediction = y_pred.max(dim=1)
        accuracy = (prediction == y_train).float().mean()
    
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Early stopping based on validation loss
        if trial.should_prune(epoch, loss.item()):
            raise optuna.exceptions.TrialPruned()
        
    return accuracy.item()

study = optuna.create_study(direction='maximize')
study.optimize(optimize, n_trials=100)

best_params = study.best_params
print('Best params: ', best_params)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import torch

# Reshape X_train to have a shape of (batch_size, sequence_length, input_dim)
X_train = torch.tensor(X_train, dtype=torch.float32).reshape(-1, 30, 258)

# Set model to evaluation mode
model.eval()

# Compute model predictions on X_train
with torch.no_grad():
    yhat = model(X_train)

# Convert predictions and true labels to NumPy arrays
yhat = yhat.numpy()
ytrue = ytrue.numpy()

# Compute confusion matrix and accuracy score
confusion_matrix = multilabel_confusion_matrix(ytrue, yhat)
accuracy = accuracy_score(ytrue, yhat)

# Print results
print(confusion_matrix)
print(accuracy)

In [ ]:
from torchsummary import summary

# Print model summary
summary(model, X_train.flatten())

In [ ]:
from sklearn.metrics import confusion_matrix
# Build the confusion matrix
cm = confusion_matrix(y_train, ytrue)

In [ ]:
X_test = preprocess_data(X_test)

# Set the model to evaluation mode
model.eval()

# Make the prediction
with torch.no_grad():
    yhat = model(X_test)

In [ ]:
# Testing the model 
y_test = torch.tensor(y_test, dtype=torch.long)
X_test = np.stack([np.array(t, dtype=np.float32) for t in X_test], axis=0)
X_test = torch.tensor(X_test, dtype=torch.float32)

with torch.no_grad():
    y_pred = model(X_test)
    loss = loss_fn(y_pred, y_test)
    _, prediction = y_pred.max(dim=1)
    accuracy = (prediction == y_test).float().mean()
    print('Test loss: ', loss.item(), 'Test accuracy: ', accuracy.item()*100, '%')
    
    # Calculate predicted class labels
    prediction_labels = [label_map[i] for i in prediction.argmax(dim=1)]
    
    # Calculate true class labels
    true_labels = [label_map[i] for i in y_test.argmax(dim=1)]
    
    # Calculate confusion matrix
    confusion_matrix = pd.crosstab(true_labels, prediction_labels, rownames=['True'], colnames=['Predicted'], margins=True)
    
    # Plot heatmap
    plt.figure(figsize=(10,7))
    sns.heatmap(confusion_matrix, annot=True, fmt='d')
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# Convert the test set labels to integers
y_test_int = y_test_np.argmax(axis=1)

# Convert the predictions to integers
predictions_int = predictions_np.argmax(axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(y_test_int, predictions_int)

print(cm)

In [ ]:
import torch

# Convert the prediction and true labels to tensors
prediction = torch.tensor(prediction, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Calculate the confusion matrix
cm = torch.zeros(len(labels), len(labels))
for t, p in zip(y_test, prediction):
    cm[t, p] += 1

# Convert the confusion matrix to a Pandas dataframe for easier visualization
cm_df = pd.DataFrame(cm.numpy(), index=labels, columns=labels)

# Create a heatmap using seaborn
sns.heatmap(cm_df, annot=True)

In [ ]:
import cv2
import numpy as np
import torch
import mediapipe.python.solutions.holistic as mp_holistic

folder = os.path.join('MP_Data') 

signs = np.array(['hello', 'sorry', 'help'])

sign_names = signs.tolist()


# Initialize the camera capture
cap = cv2.VideoCapture(0)

while True:
  # Capture a frame from the camera
  ret, frame = cap.read()

  # Preprocess the frame to extract the features
  features = preprocess_frame(frame)

  # Convert the features to a tensor and pass them through the model
  features = torch.tensor(features, dtype=torch.float32)
  prediction = model(features)
  
  # Get the index of the most likely action
  _, index = prediction.max(dim=1)
  index = index.item()
  
  # Map the index to the corresponding action name
  action_name = sign_names[index]
  
  # Display the action name on the frame
  cv2.putText(frame, action_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  
  # Show the frame
  cv2.imshow('Frame', frame)
  
  # Break out of the loop if the user presses 'q'
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

folder = os.path.join('MP_Data') 

signs = np.array(['hello', 'sorry', 'help'])

sign_names = signs.tolist()

# Initialize the camera capture
cap = cv2.VideoCapture(0)

while True:
  # Capture a frame from the camera
  ret, frame = cap.read()

  # Preprocess the frame to extract the features
  features = preprocess_frame(frame)

  # Convert the features to a tensor and pass them through the model
  features = torch.tensor(features, dtype=torch.float32)
  prediction = model(features)
  
  # Get the index of the most likely action
  _, index = prediction.max(dim=1)
  index = index.item()
  
  # Map the index to the corresponding action name
  action_name = sign_names[index]
  
  # Display the action name on the frame
  cv2.putText(frame, action_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  
  # Show the frame
  cv2.imshow('Frame', frame)
  
  # Break out of the loop if the user presses 'q'
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch



# Set up the camera
camera = cv2.VideoCapture(0)  # 0 indicates that you want to use the default camera

while True:
    # Capture a frame from the camera
    _, frame = camera.read()

    # Pre-process the frame
    keypoints = extract_keypoints(frame)  # Extract keypoints/landmarks from the frame
    keypoints = keypoints.to(torch.float32)  # Convert keypoints to a PyTorch tensor
    keypoints = keypoints.unsqueeze(0)  # Add a batch dimension to the keypoints tensor

    # Pass the keypoints through the model
    output = model(keypoints)

    # Get the index of the class with the highest probability
    _, pred = torch.max(output, dim=1)

    # Draw the prediction on the frame
    cv2.putText(frame, str(pred.item()), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    # Show the frame
    cv2.imshow('frame', frame)

    if key == ord('q'):  # If the user pressed 'q', break out of the loop
        break

# Release the camera and destroy the window
camera.release()
cv2.destroyAllWindows()